# CS4248 Project
(DON'T CLICK RUN ALL! Lemmatization takes tooo long and pls don't run it again)

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# 
# %cd "/content/drive/My Drive/CS4248 Project"
# !cd "/content/drive/My Drive/CS4248 Project"

%pip install -r requirements.txt

     ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
      --------------------------------------- 0.2/15.0 MB 5.0 MB/s eta 0:00:03
     - -------------------------------------- 0.6/15.0 MB 5.8 MB/s eta 0:00:03
     -- ------------------------------------- 1.0/15.0 MB 7.6 MB/s eta 0:00:02
     -- ------------------------------------- 1.0/15.0 MB 5.8 MB/s eta 0:00:03
     ---- ----------------------------------- 1.5/15.0 MB 6.4 MB/s eta 0:00:03
     ---- ----------------------------------- 1.8/15.0 MB 6.6 MB/s eta 0:00:03
     ----- ---------------------------------- 2.2/15.0 MB 6.7 MB/s eta 0:00:02
     ------ --------------------------------- 2.6/15.0 MB 7.1 MB/s eta 0:00:02
     -------- ------------------------------- 3.0/15.0 MB 7.1 MB/s eta 0:00:02
     --------- ------------------------------ 3.4/15.0 MB 7.2 MB/s eta 0:00:02
     ---------- ----------------------------- 3.9/15.0 MB 7.5 MB/s eta 0:00:02
     ----------- ---------------------------- 4.3/15.0 MB 7


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [13]:
import pandas as pd
import numpy as np
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import ComplementNB
from sklearn.ensemble import RandomForestClassifier

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
csv.field_size_limit(999999)
train = pd.read_csv('raw_data/fulltrain.csv', header = None, names=['class','text'])
X_train = train['text']
y_train = train['class']

In [4]:
train.head()

,class,text
0,1,"A little less than a decade ago, hockey fans w..."
1,1,The writers of the HBO series The Sopranos too...
2,1,Despite claims from the TV news outlet to offe...
3,1,After receiving 'subpar' service and experienc...
4,1,After watching his beloved Seattle Mariners pr...


In [5]:
test = pd.read_csv("raw_data/balancedtest.csv", header = None, names=['class','text'])
X_test = test['text']
y_test = test['class']

In [6]:
test.head()

,class,text
0,1,When so many actors seem content to churn out ...
1,1,In what football insiders are calling an unex...
2,1,In a freak accident following Game 3 of the N....
3,1,North Koreas official news agency announced to...
4,1,The former Alaska Governor Sarah Palin would b...


## Baseline Models

In [7]:
def tfidf(X_train, X_test):

  tfidf_baseline = TfidfVectorizer(ngram_range=(1,1), max_features = 10000)

  X_train_tfidf = tfidf_baseline.fit_transform(X_train)

  X_test_tfidf = tfidf_baseline.transform(X_test)

  return X_train_tfidf, X_test_tfidf

X_train_tfidf, X_test_tfidf = tfidf(X_train, X_test)

In [ ]:
def NB(X_train_tfidf, y_train, X_test_tfidf, y_test):

  nb_classifier = ComplementNB().fit(X_train_tfidf, y_train)

  y_pred_nb = nb_classifier.predict(X_test_tfidf)

  print(classification_report(y_test, y_pred_nb))

  return y_pred_nb

In [19]:
# NB - baseline
y_pred_nb = NB(X_train_tfidf, y_train, X_test_tfidf, y_test)

              precision    recall  f1-score   support

           1       0.62      0.61      0.61       750
           2       0.62      0.46      0.53       750
           3       0.62      0.93      0.74       750
           4       0.88      0.67      0.76       750

    accuracy                           0.67      3000
   macro avg       0.68      0.67      0.66      3000
weighted avg       0.68      0.67      0.66      3000


## Preprocessing

### Lemmatization with POS Tagging
(DON'T RUN AGAIN! It takes tooooo long)

In [73]:
# WORDNET LEMMATIZER (with appropriate pos tags)
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

# Define function to lemmatize each word with its POS tag

# POS_TAGGER_FUNCTION : TYPE 1
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatize_with_pos(tokens):
  # tokenize the sentence and find the POS tag for each token
  pos_tagged = nltk.pos_tag(tokens)

  # print(pos_tagged)

  wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
  # print(wordnet_tagged)

  lemmatized_sentence = []
  for word, tag in wordnet_tagged:
      if tag is None:
          # if there is no available tag, append the token as is
          lemmatized_sentence.append(word)
      else:
          # else use the tag to lemmatize the token
          lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
  # lemmatized_sentence = " ".join(lemmatized_sentence)

  # print(lemmatized_sentence)
  return lemmatized_sentence

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/stella/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [74]:
# Start preprocessing

from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
from tqdm import tqdm
from spellchecker import SpellChecker

punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

spell = SpellChecker()

def preprocessing(data):
  data_clean = []
  for sentence in tqdm(data):

    # Tokenization
    tokens = word_tokenize(sentence)

    # Remove punctuation and number
    tokens = [w for w in tokens if (not w in punc) and (not w.isdigit())]

    # Spell check
    # Taking too long time for each sentence, not practical to be used
    # tokens = [spell.correction(w) for w in tokens]
    # tokens = [w for w in tokens if w is not None and len(w) > 0]

    # Lemmatization based on tagging
    tokens = lemmatize_with_pos(tokens)

    data_clean.append((' ').join(tokens))
  return data_clean

X_train_clean = preprocessing(X_train)
X_test_clean = preprocessing(X_test)

[nltk_data] Downloading package punkt to /Users/stella/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/stella/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
100%|██████████| 3000/3000 [00:33<00:00, 88.88it/s] 


In [79]:
import csv
import pandas as pd

def write_to_csv(X_train_clean, X_test_clean):
    with open('raw_data/X_train_clean.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        for doc in X_train_clean:
            writer.writerow([doc])

    with open('raw_data/X_test_clean.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        for doc in X_test_clean:
            writer.writerow([doc])

# Uncomment if you want to save X_train_clean and X_test_clean
# write_to_csv(X_train_clean, X_test_clean)

In [80]:
def read_from_file():
    a_df = pd.read_csv('raw_data/X_train_clean.csv', header=None, dtype=str)
    a = a_df[0].values
    
    b_df = pd.read_csv('raw_data/X_test_clean.csv', header=None, dtype=str)
    b = b_df[0].values
    return a, b

# Uncomment if you want to read from file
a, b = read_from_file()
X_train_clean, X_test_clean = a, b

In [82]:
X_train_clean_tfidf, X_test_clean_tfidf = tfidf(X_train_clean, X_test_clean)

_ = NB(X_train_clean_tfidf, y_train, X_test_clean_tfidf, y_test)

              precision    recall  f1-score   support

           1       0.60      0.61      0.61       750
           2       0.65      0.51      0.57       750
           3       0.63      0.92      0.75       750
           4       0.87      0.64      0.74       750

    accuracy                           0.67      3000
   macro avg       0.69      0.67      0.67      3000
weighted avg       0.69      0.67      0.67      3000


### Removing stopwords, duplicates, and contradicting labels

In [83]:
def lowercase(text): #trivial casefolding
    return text.lower()

#Obtain set of stopwords from nltk
stop_words = set(stopwords.words('english'))

def remove_stopwords(data):
    data_removed_stopwords = []
    for sentence in tqdm(data):
        #tokenize sentence into words
        words = word_tokenize(sentence)
        
        #remove word if found in set of stopwords
        words_wo_stopwords = [word for word in words if word.lower() not in stop_words]
        
        data_removed_stopwords.append(' '.join(words_wo_stopwords))
    return data_removed_stopwords

def remove_dupes(data):
    #use pandas to drop duplicates, only removes those with equal values in both class and text columns
    return data.drop_duplicates()

def remove_contradictions(data):
    #group by text and filter groups with more than one unique label(contradicting labels)
    contradictions = data.groupby('text').filter(lambda x: x['class'].nunique() > 1)
    
    #remove all texts with contradicting labels since we do not know which is the correct label
    return data[~data['text'].isin(contradictions['text'])]

def preprocess2(data):
    processed_data = remove_dupes(data)
    processed_data = remove_contradictions(processed_data)
    processed_data['text'] = remove_stopwords(processed_data['text'])
    
    return processed_data

train_clean2 = preprocess2(train)    
test_clean2 = preprocess2(test)    

100%|██████████| 2990/2990 [00:04<00:00, 606.07it/s]


In [84]:
X_train_clean2 = train_clean2['text']
y_train_clean2 = train_clean2['class']
X_test_clean2 = test_clean2['text']
y_test_clean2 = test_clean2['class']

X_train_clean2_tfidf, X_test_clean2_tfidf = tfidf(X_train_clean2, X_test_clean2)

In [86]:
_ = NB(X_train_clean2_tfidf, y_train_clean2, X_test_clean2_tfidf, y_test_clean2)

              precision    recall  f1-score   support

           1       0.60      0.60      0.60       745
           2       0.61      0.45      0.52       750
           3       0.61      0.92      0.73       747
           4       0.87      0.67      0.76       748

    accuracy                           0.66      2990
   macro avg       0.67      0.66      0.65      2990
weighted avg       0.67      0.66      0.65      2990


## Feature Engineering

### Text subjectivity

In [87]:
# Text Subjectivity (w/o preprocessing)
from textblob import TextBlob

def subjectivity(data, tf_idf):
    # new_data = data.copy()
    new_data = data
    new_data['psycho-linguistic'] = new_data["text"].apply(lambda x: TextBlob(x).sentiment)
    new_data['subjectivity'] = new_data['psycho-linguistic'].apply(lambda x: x[1])
    tfidf_df = pd.DataFrame(tf_idf.toarray())
    subject_df = pd.merge(tfidf_df, new_data['subjectivity'],left_index=True, right_index=True)
    subject_df.columns = subject_df.columns.astype(str)
    return subject_df

train_subject_df = subjectivity(train, X_train_tfidf)
test_subject_df = subjectivity(test, X_test_tfidf)

In [89]:
_ = NB(train_subject_df, y_train, test_subject_df, y_test)

              precision    recall  f1-score   support

           1       0.62      0.62      0.62       750
           2       0.62      0.46      0.53       750
           3       0.62      0.93      0.74       750
           4       0.88      0.67      0.76       750

    accuracy                           0.67      3000
   macro avg       0.68      0.67      0.66      3000
weighted avg       0.68      0.67      0.66      3000


In [90]:
# Text Subjectivity (with preprocessing)
train_clean_subject_df = subjectivity(train, X_train_clean_tfidf)
test_clean_subject_df = subjectivity(test, X_test_clean_tfidf)

In [92]:
_ = NB(train_clean_subject_df, y_train, test_clean_subject_df, y_test)

              precision    recall  f1-score   support

           1       0.60      0.61      0.61       750
           2       0.64      0.51      0.57       750
           3       0.63      0.92      0.75       750
           4       0.87      0.65      0.74       750

    accuracy                           0.67      3000
   macro avg       0.69      0.67      0.67      3000
weighted avg       0.69      0.67      0.67      3000


### TF-IDF Vectorizer
We will adjust the possible parameters, i.e. n_grams, max_features here.

In [84]:
# All this need Kfold cross validation in the future

n_grams = [1, 2, 3, 4]
max_features = [10000, 20000, None]
def tfidf(X_train, X_test, n_gram, max_feature):
  tfidf_baseline = TfidfVectorizer(ngram_range=(1,n_gram), max_features = max_feature)
  X_train_tfidf = tfidf_baseline.fit_transform(X_train)
  X_test_tfidf = tfidf_baseline.transform(X_test)
  return X_train_tfidf, X_test_tfidf

for n_gram in n_grams:
  for max_feature in max_features:
    X_train_tfidf, X_test_tfidf = tfidf(X_train_clean, X_test_clean, n_gram, max_feature)
    print(f"n_gram: {n_gram}, max_feature: {max_feature}")
    num_features = X_train_tfidf.shape[1]
    if max_feature is None:
      print(f"Number of features: {num_features}")
    _ = NB(X_train_tfidf, y_train, X_test_tfidf, y_test)

# Ran until n_gram = 3 for 30min ><

n_gram: 1, max_feature: 10000
              precision    recall  f1-score   support

           1       0.85      0.81      0.83       750
           2       0.84      0.42      0.56       750
           3       0.59      0.83      0.69       750
           4       0.81      0.92      0.86       750

    accuracy                           0.74      3000
   macro avg       0.77      0.75      0.74      3000
weighted avg       0.77      0.74      0.74      3000

              precision    recall  f1-score   support

           1       0.60      0.61      0.61       750
           2       0.65      0.51      0.57       750
           3       0.63      0.92      0.75       750
           4       0.87      0.64      0.74       750

    accuracy                           0.67      3000
   macro avg       0.69      0.67      0.67      3000
weighted avg       0.69      0.67      0.67      3000

n_gram: 1, max_feature: 20000
              precision    recall  f1-score   support

           1   

### Word Embeddings
#### GloVe
- Download from https://nlp.stanford.edu/projects/glove/
- Used Wikipedia 2014 + Gigaword 5

In [9]:
def load_glove_model(glove_file_path):
    glove_model = {}
    with open(glove_file_path, 'r', encoding="utf8") as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            # embedding = np.array([float(val) for val in split_line[1:]])
            try:
                coefs = np.asarray(split_line[1:], dtype='float32')
            except ValueError:
                pass
            glove_model[word] = coefs
    return glove_model

glove_model = load_glove_model("glove.6B/glove.6B.300d.txt")


In [14]:
def preprocess_text(text):
    return word_tokenize(text.lower())

X_train_processed = X_train.apply(preprocess_text)
X_test_processed = X_test.apply(preprocess_text)

In [16]:
def document_vector(doc, model):
    # Filter out words that are not in the embedding
    embeddings = [model[word] for word in doc if word in model]
    if not embeddings:
        # If no words in the document are in the model, return a vector of zeros
        return np.zeros(next(iter(model.values())).shape)
    # Average the embeddings
    return np.mean(embeddings, axis=0)

X_train_vectors = np.array([document_vector(doc, glove_model) for doc in X_train_processed])
X_test_vectors = np.array([document_vector(doc, glove_model) for doc in X_test_processed])

In [ ]:
# Cleaned data + GloVe embeddings + Naive Bayes
# _ = NB(glove_embedding_features, y_train, glove_embedding_features_test, y_test)
# ValueError: Negative values in data passed to ComplementNB (input X)

### Hedging word count

In [98]:
# Hedging Word Count
# Reference: all 176 hedging words in hedges.txt are from https://github.com/words/hedges
import re

def fetch_hedging_regex(filename='hedges.txt'):
    hedges = []
    file = open(filename, 'r')
    lines = file.readlines()
    for l in lines:
        hedges.append(l.strip())
    hedges_re = "(?:^|\W)(" + "|".join(hedges) + ")(?:$|\W)"
    return hedges_re

def hedges(x, hedges_re):
    res = re.findall(hedges_re, x.lower())
    counts = sum(1 for s in res if s != '')
    if counts > 0:
        return 1 
    return 0

def hedging_word_count(data, tf_idf, hedge_re):
    new_data = data.copy()
    new_data["hedges"] = new_data.loc[:, "text"].apply(hedges, hedges_re=hedge_re)
    tfidf_df = pd.DataFrame(tf_idf.toarray())
    hedged_df = pd.merge(tfidf_df, new_data['hedges'],left_index=True, right_index=True)
    hedged_df.columns = hedged_df.columns.astype(str)
    return hedged_df

hedges_re = fetch_hedging_regex()
train_hedge_df = hedging_word_count(train, X_train_tfidf, hedges_re)
test_hedge_df = hedging_word_count(test, X_test_tfidf, hedges_re)

In [100]:
_ = NB(train_hedge_df, y_train, test_hedge_df, y_test)

              precision    recall  f1-score   support

           1       0.62      0.62      0.62       750
           2       0.62      0.46      0.53       750
           3       0.62      0.93      0.74       750
           4       0.88      0.67      0.76       750

    accuracy                           0.67      3000
   macro avg       0.69      0.67      0.66      3000
weighted avg       0.69      0.67      0.66      3000


In [101]:
# Hedging Word Count (with preprocessing)
train_clean_hedge_df = hedging_word_count(train, X_train_clean_tfidf, hedges_re)
test_clean_hedge_df = hedging_word_count(test, X_test_clean_tfidf, hedges_re)

In [103]:
_ = NB(train_clean_hedge_df, y_train, test_clean_hedge_df, y_test)

              precision    recall  f1-score   support

           1       0.60      0.61      0.61       750
           2       0.64      0.51      0.57       750
           3       0.63      0.91      0.75       750
           4       0.87      0.64      0.74       750

    accuracy                           0.67      3000
   macro avg       0.69      0.67      0.67      3000
weighted avg       0.69      0.67      0.67      3000


### Text Sentiment

In [83]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/james/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [84]:
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer

sia = SentimentIntensityAnalyzer()

def sentiment_features(data, tf_idf):
    new_data = data.copy()
    
    new_data['sentiments'] = new_data['text'].apply(lambda x: sia.polarity_scores(x))
    new_data['compound'] = new_data['sentiments'].apply(lambda x: x['compound'])
    tfidf_df = pd.DataFrame(tf_idf.toarray())
    sentiment_df = pd.merge(tfidf_df, new_data['compound'], left_index=True, right_index=True)
    sentiment_df.columns = sentiment_df.columns.astype(str)
    
    return sentiment_df

train_sentiment_df = sentiment_features(train, X_train_tfidf)
test_sentiment_df = sentiment_features(test, X_test_tfidf)

In [86]:
# Sentiment Scores may be negative so NB will not work. For NB, I have added a constant to the sentiment scores to make them all positive.
min_compound_train = train_sentiment_df['compound'].min()
min_compound_test = test_sentiment_df['compound'].min()
min_compound = min(min_compound_train, min_compound_test)

if min_compound < 0:
    adjustment = abs(min_compound)
    train_sentiment_df['compound'] += adjustment
    test_sentiment_df['compound'] += adjustment

_ = NB(train_sentiment_df, y_train, test_sentiment_df)

              precision    recall  f1-score   support

           1       0.61      0.64      0.62       750
           2       0.62      0.45      0.52       750
           3       0.62      0.92      0.74       750
           4       0.88      0.66      0.75       750

    accuracy                           0.66      3000
   macro avg       0.68      0.66      0.66      3000
weighted avg       0.68      0.66      0.66      3000


In [87]:
#With Preprocessing
train_clean_sentiment_df = sentiment_features(train, X_train_clean_tfidf)
test_clean_sentiment_df = sentiment_features(test, X_test_clean_tfidf)

In [89]:
# Sentiment Scores may be negative so NB will not work. For NB, I have added a constant to the sentiment scores to make them all positive.
min_clean_compound_train = train_clean_sentiment_df['compound'].min()
min_clean_compound_test = test_clean_sentiment_df['compound'].min()
min_clean_compound = min(min_clean_compound_train, min_clean_compound_test)

if min_clean_compound < 0:
    adjustment = abs(min_clean_compound)
    train_clean_sentiment_df['compound'] += adjustment
    test_clean_sentiment_df['compound'] += adjustment
    
_ = NB(train_clean_sentiment_df, y_train, test_clean_sentiment_df)

              precision    recall  f1-score   support

           1       0.60      0.62      0.61       750
           2       0.63      0.48      0.54       750
           3       0.62      0.91      0.74       750
           4       0.87      0.64      0.74       750

    accuracy                           0.66      3000
   macro avg       0.68      0.66      0.66      3000
weighted avg       0.68      0.66      0.66      3000


### Additional Features

- Sentence Length
- Number of quotations
- Count of word types:
    - Numbers
    - Proper nouns
    - Conjunctions
    - Superlatives
    - 1st / 3rd pronoun counts


In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from scipy.sparse import hstack

def add_custom_feature_to_tfidf(data, tfidf_matrix, feature_extraction_func):
    custom_feature = data.apply(feature_extraction_func)
    custom_feature_sparse = np.array(custom_feature).reshape(-1, 1)
    enhanced_tfidf_matrix = hstack([tfidf_matrix, custom_feature_sparse])
    return enhanced_tfidf_matrix

# Feature extraction functions
def count_sentence_length(text):
    return len(nltk.sent_tokenize(text))

In [ ]:
train_length_df = add_custom_feature_to_tfidf(X_train, X_train_tfidf, count_sentence_length)
test_length_df = add_custom_feature_to_tfidf(X_test, X_test_tfidf, count_sentence_length)

In [94]:
# For some reason, after scaling sentence length can give negative values so I will just use NB with unscaled sentence length.

_ = NB(train_length_df, y_train, test_length_df)

              precision    recall  f1-score   support

           1       0.65      0.86      0.74       750
           2       0.72      0.42      0.53       750
           3       0.56      0.93      0.70       750
           4       0.90      0.39      0.55       750

    accuracy                           0.65      3000
   macro avg       0.71      0.65      0.63      3000
weighted avg       0.71      0.65      0.63      3000
